#
<FONT size="10pt"> <center>API N°2 : La météo</center></FONT>. 

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import tqdm
import random

from IPython.display import display
pd.options.display.max_columns = None

# 1 : Analyse de l'API 

In [2]:
# Lien url de la request : 
Token = 'ae8525bc3ad27cd71c7b1928ef6dd750ed35fd6cd3b4dcf8692b3c91fcf558e9'
link = 'https://api.meteo-concept.com/api/forecast/daily/periods?insee=34172&start=0&end=13&token={}'.format(Token)

# Request du lien : 
r = requests.get(link)

# Vérificiation de la bonne lecture de l'API : (200 = Good) :
r

<Response [200]>

In [3]:
# Lecture de la request en json :
especes = r.json()

# Visualiser un fichier json de façon plus structuré :
Meteo = pd.json_normalize(especes)

# Data Frame :
Meteo

,update,forecast,city.insee,city.cp,city.name,city.latitude,city.longitude,city.altitude
0,2024-02-21T14:10:13+01:00,"[[{'insee': '34057', 'cp': 34170, 'latitude': ...",34172,34000,Montpellier,43.6134,3.8685,45


# 2 : Analyse des données météo sur 14 j

In [4]:
# les données météo sont dans la colonne forecast :
Met14 = pd.json_normalize(Meteo['forecast'][0])

# Modification des noms de colonnes :
Met14.columns = ['Nuit', 'Matin', 'ApresMidi', 'Soiree']

Met14.head()

,Nuit,Matin,ApresMidi,Soiree
0,"{'insee': '34057', 'cp': 34170, 'latitude': 43...","{'insee': '34057', 'cp': 34170, 'latitude': 43...","{'insee': '34057', 'cp': 34170, 'latitude': 43...","{'insee': '34057', 'cp': 34170, 'latitude': 43..."
1,"{'insee': '34057', 'cp': 34170, 'latitude': 43...","{'insee': '34057', 'cp': 34170, 'latitude': 43...","{'insee': '34057', 'cp': 34170, 'latitude': 43...","{'insee': '34057', 'cp': 34170, 'latitude': 43..."
2,"{'insee': '34057', 'cp': 34170, 'latitude': 43...","{'insee': '34057', 'cp': 34170, 'latitude': 43...","{'insee': '34057', 'cp': 34170, 'latitude': 43...","{'insee': '34057', 'cp': 34170, 'latitude': 43..."
3,"{'insee': '34057', 'cp': 34170, 'latitude': 43...","{'insee': '34057', 'cp': 34170, 'latitude': 43...","{'insee': '34057', 'cp': 34170, 'latitude': 43...","{'insee': '34057', 'cp': 34170, 'latitude': 43..."
4,"{'insee': '34057', 'cp': 34170, 'latitude': 43...","{'insee': '34057', 'cp': 34170, 'latitude': 43...","{'insee': '34057', 'cp': 34170, 'latitude': 43...","{'insee': '34057', 'cp': 34170, 'latitude': 43..."


In [5]:
# Résultats pour la nuit du J10 :
pd.DataFrame.from_dict(Met14['Nuit'][10], orient='index').T

,insee,cp,latitude,longitude,day,period,datetime,temp2m,wind10m,gust10m,dirwind10m,rr10,rr1,probarain,weather,probafrost,probafog,probawind70,probawind100,gustx
0,34057,34170,43.638,3.9127,10,0,2024-03-02T01:00:00+0100,7,15,19,336,0.2,0.2,60,40,10,0,0,0,19


## 2.1 : Exemple pour la nuit

In [6]:
Nuit = pd.json_normalize(Met14['Nuit'])
Nuit = Nuit[['period','day','temp2m','wind10m','probarain', 'rr10', 'probafrost', 'probafog', 'probawind70', 'probawind100']]
Nuit.head()

,period,day,temp2m,wind10m,probarain,rr10,probafrost,probafog,probawind70,probawind100
0,0,0,9,10,0,0.0,0,0,0,0
1,0,1,10,5,20,0.0,0,40,0,0
2,0,2,10,20,60,1.2,10,0,10,0
3,0,3,6,10,40,0.4,10,0,0,0
4,0,4,5,15,50,0.2,10,0,0,0


In [7]:
# Pivotez le DataFrame : Ex pour la nuit
NuitPivot = Nuit.melt(id_vars=['day','period'], var_name='Info', value_name='Valeur')
NuitPivot = NuitPivot.sort_values('day')
NuitPivot['day'] = NuitPivot['day']+1
NuitPivot['period'] = 'Nuit'
NuitPivot['columns'] = 'J' + NuitPivot['day'].astype(str) + '-' + NuitPivot['period'].astype(str) + '-' + NuitPivot['Info'].astype(str)
NuitPivot = NuitPivot[['columns','Valeur']]
NuitPivot.set_index('columns').T

columns,J1-Nuit-temp2m,J1-Nuit-probafrost,J1-Nuit-probarain,J1-Nuit-probafog,J1-Nuit-rr10,J1-Nuit-probawind100,J1-Nuit-probawind70,J1-Nuit-wind10m,J2-Nuit-probafrost,J2-Nuit-probarain,J2-Nuit-probafog,J2-Nuit-wind10m,J2-Nuit-probawind70,J2-Nuit-rr10,J2-Nuit-probawind100,J2-Nuit-temp2m,J3-Nuit-probafog,J3-Nuit-rr10,J3-Nuit-wind10m,J3-Nuit-probarain,J3-Nuit-probawind70,J3-Nuit-temp2m,J3-Nuit-probafrost,J3-Nuit-probawind100,J4-Nuit-probarain,J4-Nuit-temp2m,J4-Nuit-probawind70,J4-Nuit-probafog,J4-Nuit-rr10,J4-Nuit-wind10m,J4-Nuit-probawind100,J4-Nuit-probafrost,J5-Nuit-probawind100,J5-Nuit-rr10,J5-Nuit-temp2m,J5-Nuit-probafrost,J5-Nuit-wind10m,J5-Nuit-probawind70,J5-Nuit-probarain,J5-Nuit-probafog,J6-Nuit-probafrost,J6-Nuit-probafog,J6-Nuit-rr10,J6-Nuit-probawind70,J6-Nuit-probawind100,J6-Nuit-probarain,J6-Nuit-temp2m,J6-Nuit-wind10m,J7-Nuit-probafog,J7-Nuit-probafrost,J7-Nuit-probawind70,J7-Nuit-probarain,J7-Nuit-probawind100,J7-Nuit-wind10m,J7-Nuit-temp2m,J7-Nuit-rr10,J8-Nuit-wind10m,J8-Nuit-probafog,J8-Nuit-probarain,J8-Nuit-temp2m,J8-Nuit-probawind70,J8-Nuit-rr10,J8-Nuit-probafrost,J8-Nuit-probawind100,J9-Nuit-probafog,J9-Nuit-probawind100,J9-Nuit-probafrost,J9-Nuit-temp2m,J9-Nuit-rr10,J9-Nuit-wind10m,J9-Nuit-probarain,J9-Nuit-probawind70,J10-Nuit-probafog,J10-Nuit-temp2m,J10-Nuit-probawind70,J10-Nuit-rr10,J10-Nuit-wind10m,J10-Nuit-probarain,J10-Nuit-probawind100,J10-Nuit-probafrost,J11-Nuit-probafog,J11-Nuit-probarain,J11-Nuit-wind10m,J11-Nuit-rr10,J11-Nuit-probawind70,J11-Nuit-temp2m,J11-Nuit-probawind100,J11-Nuit-probafrost,J12-Nuit-temp2m,J12-Nuit-probarain,J12-Nuit-probafog,J12-Nuit-probafrost,J12-Nuit-rr10,J12-Nuit-probawind70,J12-Nuit-wind10m,J12-Nuit-probawind100,J13-Nuit-probawind100,J13-Nuit-temp2m,J13-Nuit-probawind70,J13-Nuit-probafrost,J13-Nuit-wind10m,J13-Nuit-probafog,J13-Nuit-rr10,J13-Nuit-probarain,J14-Nuit-rr10,J14-Nuit-probafog,J14-Nuit-probafrost,J14-Nuit-probarain,J14-Nuit-wind10m,J14-Nuit-temp2m,J14-Nuit-probawind70,J14-Nuit-probawind100
Valeur,9.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,20.0,40.0,5.0,0.0,0.0,0.0,10.0,0.0,1.2,20.0,60.0,10.0,10.0,10.0,0.0,40.0,6.0,0.0,0.0,0.4,10.0,0.0,10.0,0.0,0.2,5.0,10.0,15.0,0.0,50.0,0.0,0.0,0.0,2.0,10.0,0.0,60.0,8.0,15.0,0.0,10.0,10.0,60.0,0.0,25.0,7.0,3.0,20.0,0.0,60.0,6.0,10.0,1.2,10.0,0.0,0.0,0.0,10.0,7.0,0.2,20.0,60.0,10.0,0.0,7.0,0.0,0.0,15.0,50.0,0.0,10.0,0.0,60.0,15.0,0.2,0.0,7.0,0.0,10.0,8.0,60.0,0.0,0.0,1.2,10.0,15.0,0.0,0.0,9.0,10.0,0.0,20.0,0.0,2.0,60.0,0.4,0.0,0.0,60.0,20.0,8.0,10.0,0.0


## 2.2 : Pour les 4 périodes

In [8]:
def transfo(Value) :
    DataFrame= pd.json_normalize(Value)
    DataFrame= DataFrame[['period','day','temp2m','wind10m','probarain', 'rr10', 'probafrost', 'probafog', 'probawind70', 'probawind100']]
    DataFrame = DataFrame.melt(id_vars=['day','period'], var_name='Info', value_name='Valeur')
    DataFrame = DataFrame.sort_values('day')
    DataFrame['day'] = DataFrame['day']+1
    
    # faire varier la period :
    Zone = [0, 1, 2, 3]
    Plages = ['Nuit', 'Matin', 'ApresMidi', 'Soiree']
    for z, p in zip(Zone, Plages) :
        if DataFrame['period'].unique() == z :
            DataFrame['period'] = p
    DataFrame['columns'] = 'J' + DataFrame['day'].astype(str) + '-' + DataFrame['period'].astype(str) + '-' + DataFrame['Info'].astype(str)
    DataFrame = DataFrame[['columns','Valeur']]
    DataFrame = DataFrame.set_index('columns').T
    
    return DataFrame

In [9]:
# Application de la déf :
Plages = ['Nuit', 'Matin', 'ApresMidi', 'Soiree']
DFPlages = ['DFNuit', 'DFMatin', 'DFApresMidi', 'DFSoiree']

for plage, df in zip(Plages, DFPlages) :
    globals()[df] = transfo(Met14[plage]).reset_index(drop= True)

In [10]:
# Infos générales :
Meteo = Meteo[['city.insee', 'city.cp', 'city.name', 'city.latitude', 'city.longitude', 'city.altitude']]

In [11]:
# Merge des 4 périodes + infos générales :
Dataframe = pd.merge(Meteo, DFNuit,left_index = True, right_index = True)
Dataframe = pd.merge(Dataframe, DFMatin, left_index = True, right_index = True)
Dataframe = pd.merge(Dataframe, DFApresMidi, left_index = True, right_index = True)
Dataframe = pd.merge(Dataframe, DFSoiree, left_index = True, right_index = True)
Dataframe

,city.insee,city.cp,city.name,city.latitude,city.longitude,city.altitude,J1-Nuit-temp2m,J1-Nuit-probafrost,J1-Nuit-probarain,J1-Nuit-probafog,J1-Nuit-rr10,J1-Nuit-probawind100,J1-Nuit-probawind70,J1-Nuit-wind10m,J2-Nuit-probafrost,J2-Nuit-probarain,J2-Nuit-probafog,J2-Nuit-wind10m,J2-Nuit-probawind70,J2-Nuit-rr10,J2-Nuit-probawind100,J2-Nuit-temp2m,J3-Nuit-probafog,J3-Nuit-rr10,J3-Nuit-wind10m,J3-Nuit-probarain,J3-Nuit-probawind70,J3-Nuit-temp2m,J3-Nuit-probafrost,J3-Nuit-probawind100,J4-Nuit-probarain,J4-Nuit-temp2m,J4-Nuit-probawind70,J4-Nuit-probafog,J4-Nuit-rr10,J4-Nuit-wind10m,J4-Nuit-probawind100,J4-Nuit-probafrost,J5-Nuit-probawind100,J5-Nuit-rr10,J5-Nuit-temp2m,J5-Nuit-probafrost,J5-Nuit-wind10m,J5-Nuit-probawind70,J5-Nuit-probarain,J5-Nuit-probafog,J6-Nuit-probafrost,J6-Nuit-probafog,J6-Nuit-rr10,J6-Nuit-probawind70,J6-Nuit-probawind100,J6-Nuit-probarain,J6-Nuit-temp2m,J6-Nuit-wind10m,J7-Nuit-probafog,J7-Nuit-probafrost,J7-Nuit-probawind70,J7-Nuit-probarain,J7-Nuit-probawind100,J7-Nuit-wind10m,J7-Nuit-temp2m,J7-Nuit-rr10,J8-Nuit-wind10m,J8-Nuit-probafog,J8-Nuit-probarain,J8-Nuit-temp2m,J8-Nuit-probawind70,J8-Nuit-rr10,J8-Nuit-probafrost,J8-Nuit-probawind100,J9-Nuit-probafog,J9-Nuit-probawind100,J9-Nuit-probafrost,J9-Nuit-temp2m,J9-Nuit-rr10,J9-Nuit-wind10m,J9-Nuit-probarain,J9-Nuit-probawind70,J10-Nuit-probafog,J10-Nuit-temp2m,J10-Nuit-probawind70,J10-Nuit-rr10,J10-Nuit-wind10m,J10-Nuit-probarain,J10-Nuit-probawind100,J10-Nuit-probafrost,J11-Nuit-probafog,J11-Nuit-probarain,J11-Nuit-wind10m,J11-Nuit-rr10,J11-Nuit-probawind70,J11-Nuit-temp2m,J11-Nuit-probawind100,J11-Nuit-probafrost,J12-Nuit-temp2m,J12-Nuit-probarain,J12-Nuit-probafog,J12-Nuit-probafrost,J12-Nuit-rr10,J12-Nuit-probawind70,J12-Nuit-wind10m,J12-Nuit-probawind100,J13-Nuit-probawind100,J13-Nuit-temp2m,J13-Nuit-probawind70,J13-Nuit-probafrost,J13-Nuit-wind10m,J13-Nuit-probafog,J13-Nuit-rr10,J13-Nuit-probarain,J14-Nuit-rr10,J14-Nuit-probafog,J14-Nuit-probafrost,J14-Nuit-probarain,J14-Nuit-wind10m,J14-Nuit-temp2m,J14-Nuit-probawind70,J14-Nuit-probawind100,J1-Matin-temp2m,J1-Matin-probafrost,J1-Matin-probarain,J1-Matin-probafog,J1-Matin-rr10,J1-Matin-probawind100,J1-Matin-probawind70,J1-Matin-wind10m,J2-Matin-probafrost,J2-Matin-probarain,J2-Matin-probafog,J2-Matin-wind10m,J2-Matin-probawind70,J2-Matin-rr10,J2-Matin-probawind100,J2-Matin-temp2m,J3-Matin-probafog,J3-Matin-rr10,J3-Matin-wind10m,J3-Matin-probarain,J3-Matin-probawind70,J3-Matin-temp2m,J3-Matin-probafrost,J3-Matin-probawind100,J4-Matin-probarain,J4-Matin-temp2m,J4-Matin-probawind70,J4-Matin-probafog,J4-Matin-rr10,J4-Matin-wind10m,J4-Matin-probawind100,J4-Matin-probafrost,J5-Matin-probawind100,J5-Matin-rr10,J5-Matin-temp2m,J5-Matin-probafrost,J5-Matin-wind10m,J5-Matin-probawind70,J5-Matin-probarain,J5-Matin-probafog,J6-Matin-probafrost,J6-Matin-probafog,J6-Matin-rr10,J6-Matin-probawind70,J6-Matin-probawind100,J6-Matin-probarain,J6-Matin-temp2m,J6-Matin-wind10m,J7-Matin-probafog,J7-Matin-probafrost,J7-Matin-probawind70,J7-Matin-probarain,J7-Matin-probawind100,J7-Matin-wind10m,J7-Matin-temp2m,J7-Matin-rr10,J8-Matin-wind10m,J8-Matin-probafog,J8-Matin-probarain,J8-Matin-temp2m,J8-Matin-probawind70,J8-Matin-rr10,J8-Matin-probafrost,J8-Matin-probawind100,J9-Matin-probafog,J9-Matin-probawind100,J9-Matin-probafrost,J9-Matin-temp2m,J9-Matin-rr10,J9-Matin-wind10m,J9-Matin-probarain,J9-Matin-probawind70,J10-Matin-probafog,J10-Matin-temp2m,J10-Matin-probawind70,J10-Matin-rr10,J10-Matin-wind10m,J10-Matin-probarain,J10-Matin-probawind100,J10-Matin-probafrost,J11-Matin-probafog,J11-Matin-probarain,J11-Matin-wind10m,J11-Matin-rr10,J11-Matin-probawind70,J11-Matin-temp2m,J11-Matin-probawind100,J11-Matin-probafrost,J12-Matin-temp2m,J12-Matin-probarain,J12-Matin-probafog,J12-Matin-probafrost,J12-Matin-rr10,J12-Matin-probawind70,J12-Matin-wind10m,J12-Matin-probawind100,J13-Matin-probawind100,J13-Matin-temp2m,J13-Matin-probawind70,J13-Matin-probafrost,J13-Matin-wind10m,J13-Matin-probafog,J13-Matin-rr10,J13-Matin-probarai

# 3 : Résutats pour 10 villes aléatoires

In [12]:
# Création d'une DF vide:
DFFinal = pd.DataFrame()

for i in range(1, 11) : 
    
    # Réinitialise le code r a chaque boucle :
    r = None
    
    while r is None or r.status_code == 400 :
        
        # Ville aléatoire :
        ville = random.randint(0, 90000)
        
        # Token :
        Token = 'ae8525bc3ad27cd71c7b1928ef6dd750ed35fd6cd3b4dcf8692b3c91fcf558e9'
        
        # Lien url de la request : 
        link = 'https://api.meteo-concept.com/api/forecast/daily/periods?insee={}&start=0&end=13&token={}'.format(ville, Token)

        # Request du lien : 
        r = requests.get(link)

    # Lecture de la request en json :
    especes = r.json()

    # Visualiser un fichier json de façon plus structuré :
    Meteo = pd.json_normalize(especes)

    # les données météo sont dans la colonne forecast :
    Met14 = pd.json_normalize(Meteo['forecast'][0])
    
    # Modification des noms de colonnes :
    Met14.columns = ['Nuit', 'Matin', 'ApresMidi', 'Soiree']
    
    # Application de la déf :
    Plages = ['Nuit', 'Matin', 'ApresMidi', 'Soiree']
    DFPlages = ['DFNuit', 'DFMatin', 'DFApresMidi', 'DFSoiree']

    for plage, df in zip(Plages, DFPlages) :
        globals()[df] = transfo(Met14[plage]).reset_index(drop= True)
        
    # Infos générales :
    Meteo = Meteo[['city.insee', 'city.cp', 'city.name', 'city.latitude', 'city.longitude', 'city.altitude']]
    
    # Merge des 4 périodes + infos générales :
    Dataframe = pd.merge(Meteo, DFNuit,left_index = True, right_index = True)
    Dataframe = pd.merge(Dataframe, DFMatin, left_index = True, right_index = True)
    Dataframe = pd.merge(Dataframe, DFApresMidi, left_index = True, right_index = True)
    Dataframe = pd.merge(Dataframe, DFSoiree, left_index = True, right_index = True)
    
    # Concat avec les autres villes :
    DFFinal = pd.concat([DFFinal,Dataframe])

In [13]:
# Data Frame Final :
DFFinal.set_index('city.name')

,city.insee,city.cp,city.latitude,city.longitude,city.altitude,J1-Nuit-temp2m,J1-Nuit-probafrost,J1-Nuit-probarain,J1-Nuit-probafog,J1-Nuit-rr10,J1-Nuit-probawind100,J1-Nuit-probawind70,J1-Nuit-wind10m,J2-Nuit-probafrost,J2-Nuit-probarain,J2-Nuit-probafog,J2-Nuit-wind10m,J2-Nuit-probawind70,J2-Nuit-rr10,J2-Nuit-probawind100,J2-Nuit-temp2m,J3-Nuit-probafog,J3-Nuit-rr10,J3-Nuit-wind10m,J3-Nuit-probarain,J3-Nuit-probawind70,J3-Nuit-temp2m,J3-Nuit-probafrost,J3-Nuit-probawind100,J4-Nuit-probarain,J4-Nuit-temp2m,J4-Nuit-probawind70,J4-Nuit-probafog,J4-Nuit-rr10,J4-Nuit-wind10m,J4-Nuit-probawind100,J4-Nuit-probafrost,J5-Nuit-probawind100,J5-Nuit-rr10,J5-Nuit-temp2m,J5-Nuit-probafrost,J5-Nuit-wind10m,J5-Nuit-probawind70,J5-Nuit-probarain,J5-Nuit-probafog,J6-Nuit-probafrost,J6-Nuit-probafog,J6-Nuit-rr10,J6-Nuit-probawind70,J6-Nuit-probawind100,J6-Nuit-probarain,J6-Nuit-temp2m,J6-Nuit-wind10m,J7-Nuit-probafog,J7-Nuit-probafrost,J7-Nuit-probawind70,J7-Nuit-probarain,J7-Nuit-probawind100,J7-Nuit-wind10m,J7-Nuit-temp2m,J7-Nuit-rr10,J8-Nuit-wind10m,J8-Nuit-probafog,J8-Nuit-probarain,J8-Nuit-temp2m,J8-Nuit-probawind70,J8-Nuit-rr10,J8-Nuit-probafrost,J8-Nuit-probawind100,J9-Nuit-probafog,J9-Nuit-probawind100,J9-Nuit-probafrost,J9-Nuit-temp2m,J9-Nuit-rr10,J9-Nuit-wind10m,J9-Nuit-probarain,J9-Nuit-probawind70,J10-Nuit-probafog,J10-Nuit-temp2m,J10-Nuit-probawind70,J10-Nuit-rr10,J10-Nuit-wind10m,J10-Nuit-probarain,J10-Nuit-probawind100,J10-Nuit-probafrost,J11-Nuit-probafog,J11-Nuit-probarain,J11-Nuit-wind10m,J11-Nuit-rr10,J11-Nuit-probawind70,J11-Nuit-temp2m,J11-Nuit-probawind100,J11-Nuit-probafrost,J12-Nuit-temp2m,J12-Nuit-probarain,J12-Nuit-probafog,J12-Nuit-probafrost,J12-Nuit-rr10,J12-Nuit-probawind70,J12-Nuit-wind10m,J12-Nuit-probawind100,J13-Nuit-probawind100,J13-Nuit-temp2m,J13-Nuit-probawind70,J13-Nuit-probafrost,J13-Nuit-wind10m,J13-Nuit-probafog,J13-Nuit-rr10,J13-Nuit-probarain,J14-Nuit-rr10,J14-Nuit-probafog,J14-Nuit-probafrost,J14-Nuit-probarain,J14-Nuit-wind10m,J14-Nuit-temp2m,J14-Nuit-probawind70,J14-Nuit-probawind100,J1-Matin-temp2m,J1-Matin-probafrost,J1-Matin-probarain,J1-Matin-probafog,J1-Matin-rr10,J1-Matin-probawind100,J1-Matin-probawind70,J1-Matin-wind10m,J2-Matin-probafrost,J2-Matin-probarain,J2-Matin-probafog,J2-Matin-wind10m,J2-Matin-probawind70,J2-Matin-rr10,J2-Matin-probawind100,J2-Matin-temp2m,J3-Matin-probafog,J3-Matin-rr10,J3-Matin-wind10m,J3-Matin-probarain,J3-Matin-probawind70,J3-Matin-temp2m,J3-Matin-probafrost,J3-Matin-probawind100,J4-Matin-probarain,J4-Matin-temp2m,J4-Matin-probawind70,J4-Matin-probafog,J4-Matin-rr10,J4-Matin-wind10m,J4-Matin-probawind100,J4-Matin-probafrost,J5-Matin-probawind100,J5-Matin-rr10,J5-Matin-temp2m,J5-Matin-probafrost,J5-Matin-wind10m,J5-Matin-probawind70,J5-Matin-probarain,J5-Matin-probafog,J6-Matin-probafrost,J6-Matin-probafog,J6-Matin-rr10,J6-Matin-probawind70,J6-Matin-probawind100,J6-Matin-probarain,J6-Matin-temp2m,J6-Matin-wind10m,J7-Matin-probafog,J7-Matin-probafrost,J7-Matin-probawind70,J7-Matin-probarain,J7-Matin-probawind100,J7-Matin-wind10m,J7-Matin-temp2m,J7-Matin-rr10,J8-Matin-wind10m,J8-Matin-probafog,J8-Matin-probarain,J8-Matin-temp2m,J8-Matin-probawind70,J8-Matin-rr10,J8-Matin-probafrost,J8-Matin-probawind100,J9-Matin-probafog,J9-Matin-probawind100,J9-Matin-probafrost,J9-Matin-temp2m,J9-Matin-rr10,J9-Matin-wind10m,J9-Matin-probarain,J9-Matin-probawind70,J10-Matin-probafog,J10-Matin-temp2m,J10-Matin-probawind70,J10-Matin-rr10,J10-Matin-wind10m,J10-Matin-probarain,J10-Matin-probawind100,J10-Matin-probafrost,J11-Matin-probafog,J11-Matin-probarain,J11-Matin-wind10m,J11-Matin-rr10,J11-Matin-probawind70,J11-Matin-temp2m,J11-Matin-probawind100,J11-Matin-probafrost,J12-Matin-temp2m,J12-Matin-probarain,J12-Matin-probafog,J12-Matin-probafrost,J12-Matin-rr10,J12-Matin-probawind70,J12-Matin-wind10m,J12-Matin-probawind100,J13-Matin-probawind100,J13-Matin-temp2m,J13-Matin-probawind70,J13-Matin-probafrost,J13-Matin-wind10m,J13-Matin-probafog,J13-Matin-rr10,J13-Matin-probarain,J14-Mati